In [42]:
#importing the libraries
import json
import ast
import isodate
import pandas as pd
import pycountry
import plotly.express as px
template = 'plotly_dark'
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

In [43]:
#loading the csv file to python dataframe
df = pd.read_csv('youtube_data_preprocessed.csv')

In [44]:
#printing a sample of the dataframe
df.head()

,id,video_name,channel_name,channel_id,date,duration,region_code,region_name,category,statistics,channel_stat
0,1,ملخص مباراة البرتغال والأوروغواي - البرتغال ته...,beIN SPORTS,UCJUCcJUeh0Cz2xyKwkw5Q1w,2022-11-28T21:28:44Z,PT4M56S,AE,United Arab Emirates,17,"{'viewCount': '2581021', 'likeCount': '49702',...","{'viewCount': '662059394', 'subscriberCount': ..."
1,2,ITZY “Cheshire” M/V @ITZY,JYP Entertainment,UCaO6TYtlC8U5ttz62hTrZgg,2022-11-30T08:58:09Z,PT3M8S,AE,United Arab Emirates,10,"{'viewCount': '3375142', 'likeCount': '537367'...","{'viewCount': '18109940685', 'subscriberCount'..."
2,3,تحدي الثلاثين نسخة كأس العالم|كلاسيكو عبدالله ...,M12 ALFOUZAN - قدم الكرة,UCqe2V_eejw-b5SeOw63TQNw,2022-11-30T10:00:04Z,PT28M42S,AE,United Arab Emirates,17,"{'viewCount': '282450', 'likeCount': '44190', ...","{'viewCount': '70894383', 'subscriberCount': '..."
3,4,ملخص مباراة ويلز وإنكلترا | المنتخب الإنكليزي ...,beIN SPORTS,UCJUCcJUeh0Cz2xyKwkw5Q1w,2022-11-29T21:23:19Z,PT5M2S,AE,United Arab Emirates,17,"{'viewCount': '1388618', 'likeCount': '27587',...","{'viewCount': '662059394', 'subscriberCount': ..."
4,5,الكورة مع السلامة | الحلقة الثانية - كولومبيا 🇨🇴,Da7ee7 - الدحيح,UCqW7G8SmyeEeQYzLOk5tdSg,2022-11-29T18:01:24Z,PT26M50S,AE,United Arab Emirates,28,"{'viewCount': '372223', 'likeCount': '38760', ...","{'viewCount': '34045688', 'subscriberCount': '..."


In [45]:
#renaming the columns.
df.rename(columns={'category':'category_Id'}, inplace=True)

#printing the number of rows and columns before preprocessing
df.shape

(5450, 11)

In [46]:
#Loading the category json data
with open('Category_json.json') as f:
    category_json = json.load(f)

#extracting category data from json to dictionary using dict comprehension
category_list = {category_json['items'][i]['id'] : category_json['items'][i]['snippet']['title'] for i in range(len(category_json['items']))}
#mapping category code and name from the dictionary to the dataframe
df['category'] = df['category_Id'].apply(str).map(category_list)

In [47]:
#segregating data from the statistics
#extracting viewcount and converting to numerical type for dataframe
df['view'] = [ast.literal_eval(i).get('viewCount') for i in df['statistics']]
df['view'] = pd.to_numeric(df['view'])

#extracting likecount and converting to numerical type for dataframe
df['like'] = [ast.literal_eval(i).get('likeCount') for i in df['statistics']]
df['like'] = pd.to_numeric(df['like'])

#extracting commentcount and converting to numerical type for dataframe
df['comment'] = [ast.literal_eval(i).get('commentCount') for i in df['statistics']]
df['comment'] = pd.to_numeric(df['comment'])

#segregating data from the channel statistics
#extracting subscribercount and converting to numerical type for dataframe
df['channel_subscribers'] = [ast.literal_eval(i).get('subscriberCount') for i in df['channel_stat']]
df['channel_subscribers'] = pd.to_numeric(df['channel_subscribers'])

#extracting channel videocount and converting to numerical type for dataframe
df['channel_videoCount'] = [ast.literal_eval(i).get('videoCount') for i in df['channel_stat']]
df['channel_videoCount'] = pd.to_numeric(df['channel_videoCount'])

In [48]:
#printing thecolumn types of dataframe
df.dtypes

id                       int64
video_name              object
channel_name            object
channel_id              object
date                    object
duration                object
region_code             object
region_name             object
category_Id              int64
statistics              object
channel_stat            object
category                object
view                     int64
like                   float64
comment                float64
channel_subscribers      int64
channel_videoCount       int64
dtype: object

In [49]:
#printing the null values sum by columns
df.isnull().sum()

id                      0
video_name              0
channel_name            0
channel_id              0
date                    0
duration                0
region_code             0
region_name             0
category_Id             0
statistics              0
channel_stat            0
category                0
view                    0
like                   28
comment                95
channel_subscribers     0
channel_videoCount      0
dtype: int64

In [50]:
#Interchanging the Nan values to 0
df.fillna(0,inplace=True)

In [51]:
#calculating the Popularity, engagement and growth
df['popularity_%'] = round((df['like']/df['view']) * 100,2)
df['engagement_%'] = round((df['comment']/df['view']) * 100,2)
df['growth_%'] = round((df['view']/df['channel_subscribers']) * 100,2)

In [52]:
#function to extract date from the datetime object
def date_ext(text):
    return text.date()

#converting the object type to datetime type
df['date'] = pd.to_datetime(df['date'])

#Applying the date extraction function to the date column in dataframe
df['date'] = df['date'].apply(date_ext)

In [53]:
#funtion for calculating duration in seconds from youtube duration format
def duration_ext(text):
    return int(isodate.parse_duration(text).total_seconds())

#Applying the duration in seconds function to the duration column in dataframe
df['duration(sec)'] = df['duration'].apply(duration_ext)

#function for fixing the country name to extract the country code name.
def country_fix(title):
    if 'laos' in title.lower():
        return "Lao People's Democratic Republic"
    elif 'vietnam' in title.lower():
        return 'Viet Nam'
    elif 'russia' in title.lower():
        return 'Russian Federation'
    elif 'south korea' in title.lower():
        return 'Korea, Republic of'
    elif 'taiwan' in title.lower():
        return 'Taiwan, Province of China'
    elif 'bolivia' in title.lower():
        return 'Bolivia, Plurinational State of'
    elif 'tanzania' in title.lower():
        return 'Tanzania, United Republic of'
    elif 'venezuela' in title.lower():
        return 'Venezuela, Bolivarian Republic of'
    else:
        return title

#Applying the country fix function to the region name column in dataframe
df['country'] = df['region_name'].apply(country_fix)

#init a dictionary for country name and country code
countries = {}
#init the for loop to get the country code
for country in pycountry.countries:
    countries[country.name] = country.alpha_3
#list comprehension to map country code to the country column in dataframe
df['country_code'] = [countries.get(country) for country in df['country']]

In [54]:
#printing the sample of the dataframe
df.head()

,id,video_name,channel_name,channel_id,date,duration,region_code,region_name,category_Id,statistics,channel_stat,category,view,like,comment,channel_subscribers,channel_videoCount,popularity_%,engagement_%,growth_%,duration(sec)
0,1,ملخص مباراة البرتغال والأوروغواي - البرتغال ته...,beIN SPORTS,UCJUCcJUeh0Cz2xyKwkw5Q1w,2022-11-28,PT4M56S,AE,United Arab Emirates,17,"{'viewCount': '2581021', 'likeCount': '49702',...","{'viewCount': '662059394', 'subscriberCount': ...",Sports,2581021,49702.0,1324.0,8500000,3913,1.93,0.05,30.36,296
1,2,ITZY “Cheshire” M/V @ITZY,JYP Entertainment,UCaO6TYtlC8U5ttz62hTrZgg,2022-11-30,PT3M8S,AE,United Arab Emirates,10,"{'viewCount': '3375142', 'likeCount': '537367'...","{'viewCount': '18109940685', 'subscriberCount'...",Music,3375142,537367.0,30969.0,26000000,1592,15.92,0.92,12.98,188
2,3,تحدي الثلاثين نسخة كأس العالم|كلاسيكو عبدالله ...,M12 ALFOUZAN - قدم الكرة,UCqe2V_eejw-b5SeOw63TQNw,2022-11-30,PT28M42S,AE,United Arab Emirates,17,"{'viewCount': '282450', 'likeCount': '44190', ...","{'viewCount': '70894383', 'subscriberCount': '...",Sports,282450,44190.0,667.0,1120000,65,15.65,0.24,25.22,1722
3,4,ملخص مباراة ويلز وإنكلترا | المنتخب الإنكليزي ...,beIN SPORTS,UCJUCcJUeh0Cz2xyKwkw5Q1w,2022-11-29,PT5M2S,AE,United Arab Emirates,17,"{'viewCount': '1388618', 'likeCount': '27587',...","{'viewCount': '662059394', 'subscriberCount': ...",Sports,1388618,27587.0,896.0,8500000,3913,1.99,0.06,16.34,302
4,5,الكورة مع السلامة | الحلقة الثانية - كولومبيا 🇨🇴,Da7ee7 - الدحيح,UCqW7G8SmyeEeQYzLOk5tdSg,2022-11-29,PT26M50S,AE,United Arab Emirates,28,"{'viewCount': '372223', 'likeCount': '38760', ...","{'viewCount': '34045688', 'subscriberCount': '...",Science & Technology,372223,38760.0,1323.0,1530000,42,10.41,0.36,24.33,1610


In [55]:
#droping the unwanted columns
df.drop(columns=['id','statistics','channel_stat','duration'], inplace=True)

In [56]:
#checking the columns in the dataframe
df.columns

Index(['video_name', 'channel_name', 'channel_id', 'date', 'region_code',
       'region_name', 'category_Id', 'category', 'view', 'like', 'comment',
       'channel_subscribers', 'channel_videoCount', 'popularity_%',
       'engagement_%', 'growth_%', 'duration(sec)'],
      dtype='object')

In [41]:
#saving the post processed dataframe to a csv file
df.to_csv('youtube_data_postprocessed.csv', index=False)

# Geo Plotly

In [57]:
youtube_stat = df.groupby(['region_name']).agg({'view': 'mean', 'like': 'mean', 'comment': 'mean', 'popularity_%': 'mean', 'engagement_%': 'mean', 'category': 'nunique'}).reset_index().sort_values(by=['view'], ascending=False)
youtube_stat.rename(columns={'region_name':'Country', 'view':'View_Avg', 'like':'Like_Avg', 'comment':'Comment_Avg', 'popularity_%':'Popularity_Avg', 'engagement_%':'Engagement_Avg', 'category':'Category_Unique_Count'}, inplace=True)

In [58]:
youtube_stat

,Country,View_Avg,Like_Avg,Comment_Avg,Popularity_Avg,Engagement_Avg,Category_Unique_Count
13,Cambodia,26952005.28,868374.76,3607.52,2.9568,0.0366,9
57,Malaysia,18010288.16,809951.06,12659.88,4.1960,0.1624,11
50,Laos,14954699.86,429183.70,1807.86,2.3452,0.0624,8
70,Pakistan,12828527.82,583265.36,1944.54,4.0544,0.0682,8
58,Malta,11661577.28,469749.06,20792.94,3.8274,0.1864,10
...,...,...,...,...,...,...,...
94,Taiwan,827419.70,35321.22,2270.44,3.1434,0.2436,10
28,France,768793.68,55211.36,3517.18,7.0656,0.4534,11
95,Tanzania,717644.14,15705.76,782.24,2.0322,0.2812,6
81,Russia,694103.86,53934.46,2679.32,7.6856,0.4378,11


In [59]:
countries = {}
for country in pycountry.countries:
    countries[country.name] = country.alpha_3
youtube_stat['Alpha3'] = [countries.get(country) for country in youtube_stat['Country']]

In [60]:
def nationality_fix(title):
    if 'laos' in title.lower():
        return "Lao People's Democratic Republic"
    elif 'vietnam' in title.lower():
        return 'Viet Nam'
    elif 'russia' in title.lower():
        return 'Russian Federation'
    elif 'south korea' in title.lower():
        return 'Korea, Republic of'
    elif 'taiwan' in title.lower():
        return 'Taiwan, Province of China'
    elif 'bolivia' in title.lower():
        return 'Bolivia, Plurinational State of'
    elif 'tanzania' in title.lower():
        return 'Tanzania, United Republic of'
    elif 'venezuela' in title.lower():
        return 'Venezuela, Bolivarian Republic of'
    else:
        return title

In [61]:
youtube_stat['Country'] = youtube_stat['Country'].apply(nationality_fix)

In [62]:
youtube_stat

,Country,View_Avg,Like_Avg,Comment_Avg,Popularity_Avg,Engagement_Avg,Category_Unique_Count,Alpha3
13,Cambodia,26952005.28,868374.76,3607.52,2.9568,0.0366,9,KHM
57,Malaysia,18010288.16,809951.06,12659.88,4.1960,0.1624,11,MYS
50,Lao People's Democratic Republic,14954699.86,429183.70,1807.86,2.3452,0.0624,8,None
70,Pakistan,12828527.82,583265.36,1944.54,4.0544,0.0682,8,PAK
58,Malta,11661577.28,469749.06,20792.94,3.8274,0.1864,10,MLT
...,...,...,...,...,...,...,...,...
94,"Taiwan, Province of China",827419.70,35321.22,2270.44,3.1434,0.2436,10,None
28,France,768793.68,55211.36,3517.18,7.0656,0.4534,11,FRA
95,"Tanzania, United Republic of",717644.14,15705.76,782.24,2.0322,0.2812,6,None
81,Russian Federation,694103.86,53934.46,2679.32,7.6856,0.4378,11,None


In [63]:
fig = px.scatter_geo(youtube_stat, locations="Alpha3", color="Country", template= template,
                     hover_name="Country", size="View_Avg", hover_data = ['Like_Avg', 'Comment_Avg', 'Popularity_Avg', 'Engagement_Avg', 'Category_Unique_Count'],
                     projection="natural earth")
fig.show()